In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
prod_filepath = "./data/production_data.csv"
df_prod = pd.read_csv(prod_filepath)

alias_filepath = "./data/operator_aliasing.csv"
df_alias = pd.read_csv(alias_filepath)

wells_filepath = "./data/wellsData.csv"
df_wells = pd.read_csv(wells_filepath)

C:\Users\Mariam\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (21,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Adjusting API by adding a leading zero to APIs starting with '5'

In [3]:
start = time.time()
df_prod.API=df_prod.API.astype(str).str.zfill(14)
df_wells.API=df_wells.API.astype(str).str.zfill(14)

Add Operator Alias

In [4]:
filter_col = np.append(df_prod.columns, ['Alias'])

# wells_alias_df = df_alias[['Name', 'Alias']].merge(df_wells[['operatorNameIHS', 'API']], left_on='Name', right_on='operatorNameIHS', how='outer')
new_df = (df_prod.merge(df_alias[['Name', 'Alias']].merge(df_wells[['operatorNameIHS', 'API']], left_on='Name', right_on='operatorNameIHS', how='right')[['operatorNameIHS', 'Alias', 'API']], on='API', how='left'))

new_df = new_df.fillna({'Alias':new_df.operatorNameIHS})[filter_col]

Remove Pre-Peak Months

In [5]:
# filtering out post-peak records with 0 oil production
# tmp_df = new_df[filter_col][new_df['Liquid']!=0]

# getting peak records
peak_records = new_df[new_df.groupby(['API'], sort=False)['Liquid'].transform(max) == new_df['Liquid']]

# filtering out pre-peak months
peak_records = peak_records.rename(columns={"Month": "PeakMonth", "Year": "PeakYear"})
new_df = pd.merge(new_df, peak_records[["API", "PeakMonth", "PeakYear"]], on='API', how='left')
post_peak = new_df[np.logical_or((new_df['Year'] > new_df['PeakYear']),np.logical_and((new_df['Year']==new_df['PeakYear']),(new_df['Month']>=new_df['PeakMonth'])))].sort_values(['Year','Month'])[filter_col]




Add a Month Index Column

In [6]:
post_peak['MonthIdx'] = post_peak.groupby(['API'], sort=False).cumcount()+1
end = time.time()
tt = end - start
print(tt)

51.181432247161865


In [8]:
post_peak.to_csv("./data/processed_production_data.csv")

PermissionError: [Errno 13] Permission denied